In [73]:
import pulp
import pandas as pd
import numpy as np 
import json

In [74]:
# Load the distance matrix from an Excel file
excel_file = "distance_matrix.xlsx"  
dfs = pd.read_excel(excel_file, index_col=0, engine='openpyxl')

df = dfs.copy()

demand_i = np.random.rand(df.shape[1])*10000 #demand aslinya 
demands = {city : demand for city, demand in zip(df.columns,demand_i)}

In [75]:
#Transform data ke list
facilities = df.columns.tolist()  # Facility locations
demand_points = df.index.tolist()  # Demand points

#HYPER PARAMETER YANG DIPAKE (OTAK ATIK 2 VARIABLE DIBAWAH INI BUAT NAIKIN PERFORMANCENYA)
k = 10  # Number of facilities to open
maximum_coverage = 10000

In [76]:
# Inisiasi a PuLP linear programming problem
problem = pulp.LpProblem("MCLP", pulp.LpMaximize)

In [77]:
#Defining Decision Variable yang dipake 
# Define binary variables for facility opening decision
x_facility_open = {j: pulp.LpVariable("x " + str(j), cat=pulp.LpBinary) for j in facilities}   #x
y_demand_point = {i: pulp.LpVariable("y " + str(i), cat=pulp.LpBinary) for i in demand_points}  #y


In [78]:
# Define the objective function (maximize coverage)
problem += pulp.lpSum([demands[d] * y_demand_point[d] for d in y_demand_point])


In [79]:
# Constraints Declaration

# Set up constraints K (maximal SPKLU YANG DIBANGUN)
problem += pulp.lpSum([x_facility_open[f] for f in facilities]) == k

#set up constraint lokasi yang tercover 

#subset kota yang tercover 
N = {i:[j for j in facilities if df[i][j] <= 10000] for i in demand_points}
for i in demand_points:
    problem += pulp.lpSum([x_facility_open[j] for j in N[i]]) >= y_demand_point[i]

In [80]:
#Final Problem 
problem

MCLP:
MAXIMIZE
654.2836879256888*y_Bandongan + 247.947933285915*y_Bayat + 99.61082710671721*y_Borobudur + 644.4105653305903*y_Candimulyo + 712.1928272986012*y_Cawas + 767.9104688408756*y_Ceper + 993.6829865538366*y_Delanggu + 290.521110395064*y_Dukun + 822.3840686447763*y_Gantiwarno + 137.16154700536597*y_Grabag + 512.6164229752658*y_Jatinom + 27.907122491025117*y_Jogonalan + 659.7976162152607*y_Juwiring + 672.6777180202381*y_KAPANEWON_BAMBANG_LIPURO + 761.8049923769274*y_KAPANEWON_BANGUNTAPAN + 460.8961715640544*y_KAPANEWON_BANTUL + 771.6313773539978*y_KAPANEWON_BERBAH + 443.51551033107427*y_KAPANEWON_CANGKRINGAN + 307.5585128674946*y_KAPANEWON_DEPOK + 513.7037242387192*y_KAPANEWON_DLINGO + 648.44740021138*y_KAPANEWON_GALUR + 599.6646500720202*y_KAPANEWON_GAMPING + 342.8526781615868*y_KAPANEWON_GEDANGSARI + 989.4664942191175*y_KAPANEWON_GIRIMULYO + 640.6278201108407*y_KAPANEWON_GIRISUBO + 768.1086219206475*y_KAPANEWON_GODEAN + 94.01277912676642*y_KAPANEWON_IMOGIRI + 482.471938144004*y

In [81]:
problem.solve()

1

In [82]:
x_soln = np.array([x_facility_open[j].varValue for j in facilities])
x_soln

y_soln = np.array([y_demand_point[j].varValue for j in demand_points])
# And print some output
print (("Status:"), pulp.LpStatus[problem.status])
print ("Population Served is = ", pulp.value(problem.objective))
print("total demand : ", sum(list(demands.values())))
print ("x = ", x_soln)
print("y = ", y_soln)

Status: Optimal
Population Served is =  19903.693944827002
total demand :  64044.93802746409
x =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 0.]


In [85]:
#Import data to Textfiles 
facilities_opened=[]
cities_covered = []
cities_not_covered = []
for i in range(len(x_soln)):
    if x_soln[i] == 1:
        facilities_opened.append(df.columns[i])

for i in range(len(y_soln)):
    if y_soln[i] == 1:
        cities_covered.append(df.columns[i])
    else:
        cities_not_covered.append(df.columns[i])

final_res={
    'f_open':facilities_opened,
    'c_cover':cities_covered,
    'c_not_cover':cities_not_covered,
    'pop_served': pulp.value(problem.objective),
    'all_demand': sum(list(demands.values())),
    'performance':f'{pulp.value(problem.objective)/sum(list(demands.values()))} dari seluruh demand tercover'  
}

print(final_res)

with open ('hasil_akhir.txt','w') as textfiles:
    textfiles.write(json.dumps(final_res))


{'f_open': ['KAPANEWON BAMBANG LIPURO', 'KEMANTREN DANUREJAN', 'Ceper'], 'c_cover': ['KAPANEWON GALUR', 'KAPANEWON LENDAH', 'KAPANEWON SRANDAKAN', 'KAPANEWON SANDEN', 'KAPANEWON KRETEK', 'KAPANEWON PUNDONG', 'KAPANEWON BAMBANG LIPURO', 'KAPANEWON PANDAK', 'KAPANEWON BANTUL', 'KAPANEWON BANGUNTAPAN', 'KAPANEWON SEWON', 'KAPANEWON KASIHAN', 'KAPANEWON GAMPING', 'KAPANEWON GODEAN', 'KAPANEWON DEPOK', 'KAPANEWON BERBAH', 'KAPANEWON NGAGLIK', 'KEMANTREN TEGALREJO', 'KEMANTREN JETIS', 'KEMANTREN GONDOKUSUMAN', 'KEMANTREN DANUREJAN', 'KEMANTREN GEDONGTENGEN', 'KEMANTREN NGAMPILAN', 'KEMANTREN WIROBRAJAN', 'KEMANTREN MANTRIJERON', 'KEMANTREN KRATON', 'KEMANTREN GONDOMANAN', 'KEMANTREN PAKUALAMAN', 'KEMANTREN MERGANGSAN', 'KEMANTREN UMBULHARJO', 'KEMANTREN KOTAGEDE', 'Ceper', 'Delanggu', 'Kalikotes', 'Karanganom', 'Klaten Tengah', 'Klaten Utara', 'Ngawen', 'Pedan', 'Polanharjo', 'Trucuk'], 'c_not_cover': ['KAPANEWON TEMON', 'KAPANEWON WATES', 'KAPANEWON PANJATAN', 'KAPANEWON SENTOLO', 'KAPANEWO